# A simple neural network in julia
Based on Andrew Ng's Machine Learning course in Coursera

## loading data

In [ ]:
using MNIST # hand written digits

trainX, trainY = traindata();
testX, testY = testdata();

In [ ]:
# limit the dataset for demonstration purpose
trainX = trainX[:,1:5000]
trainY = trainY[1:5000]

testX = testX[:,1:1000]
testY = testY[1:1000]

trainY = round(Int, trainY) + 1 # convert to integer and allow indexing later
testY = round(Int, testY) + 1 # convert to integer and allow indexing later

print("Number of features: ", size(trainX,1), " (28x28 grey scale image)")
print("\n")
print("Number of training examples: ", size(trainX,2))
print("\n")
print("Number of testing examples: ", size(testX,2))

##visualization

In [ ]:
# display random images from the training set
using PyPlot;

for i = 1:25
    PyPlot.subplot(5,5,i)
    PyPlot.axis("off")
    PyPlot.imshow(reshape(trainX'[rand(1:size(trainX',1)),:],28,28), cmap=PyPlot.ColorMap("gray"))
end

##adding bias

In [ ]:
m = size(trainX,2) # number of training examples
X = trainX'
y = trainY'
a1 = [ones(m, 1) X] # adding bias units set to 1

mtest = size(testX,2) # number of testing examples
Xtest = testX'
ytest = testY'
a1test = [ones(mtest, 1) Xtest] # adding bias units set to 1

##model

In [ ]:
input_layer_size = size(X,2) #input layer
hidden_layer_size = 20 # hidden layer
num_labels = 10 # output layer, number of digits for classification, 0-9

lambda = 1 # regularization parameter
iterations = 10

In [ ]:
# prepare target for multiclass classification
Y = zeros(m,num_labels)
for i = 1:m
    Y[i, y[i]] = 1
end

Ytest = zeros(mtest,num_labels)
for i = 1:mtest
    Ytest[i, ytest[i]] = 1
end

In [ ]:
#initialize weights between input and hidden layers (Theta1), and between hidden and output layers (Theta2)
epsilon_init = 0.12;
Theta1 = rand(hidden_layer_size,input_layer_size+1) * 2 * epsilon_init - epsilon_init
Theta2 = rand(num_labels,hidden_layer_size+1) * 2 * epsilon_init - epsilon_init
Theta = vcat(Theta1[:], Theta2[:]) #unroll weights as required by optimization algorithm

In [ ]:
zigmoid(z) =  1.0 ./ (1.0 + exp(-z)); # sigmoid function

In [ ]:
gizmoid(z) = zigmoid(z).*(1-zigmoid(z)); # derivative of sigmoid function

In [ ]:
# cost function
function nnCostFn(Theta)
    Theta1 = reshape(Theta[1:hidden_layer_size * (input_layer_size + 1)], hidden_layer_size, (input_layer_size + 1))
    Theta2 = reshape(Theta[(1 + (hidden_layer_size * (input_layer_size + 1))):end], num_labels, (hidden_layer_size + 1))

    # forward pass
    z2 = *(Theta1, a1') # input to hidden layer
    a2 = zigmoid(z2) # hidden layer activations
    a2 = [ones(m) a2'] # adding bias units
    z3 = *(Theta2, a2') # input to output layer
    a3 = zigmoid(z3') # output layer activations
    
    J =  1/m * sum((-Y .* log(a3))  -   ((1 - Y) .* (log(1 - a3)))) # cross-entropy cost function
    Theta1Sq = Theta1[:,2:end].^2 # exclude bias
    Theta2Sq = Theta2[:,2:end].^2 # exclude bias
    J = J + lambda/(2*m) * (sum(Theta1Sq) +sum(Theta2Sq)) # regularized cost function
end

In [ ]:
# calculating gradients
function gradient(Theta)
    Theta1 = reshape(Theta[1:hidden_layer_size * (input_layer_size + 1)], hidden_layer_size, (input_layer_size + 1));
    Theta2 = reshape(Theta[(1 + (hidden_layer_size * (input_layer_size + 1))):end], num_labels, (hidden_layer_size + 1));

    # forward pass
    z2 = *(Theta1, a1') # input to hidden layer
    a2 = zigmoid(z2) # hidden layer activations
    a2 = [ones(m) a2'] # adding bias units
    z3 = *(Theta2, a2') # input to output layer
    a3 = zigmoid(z3') # output layer activations  
    
    # transpose matrices
    a1t = a1';
    a2t = a2';
    a3t = a3';
    Yt  = Y';

    # backpropagation
    # calculate errors
    delta3 = a3t - Yt;
    z2t = vcat(ones(1,m), *(Theta1,a1t))
    delta2 = *(Theta2',delta3).*gizmoid(z2t);

    # calculate gradients
    Theta1_grad = *(delta2[2:end,:],a1t');
    Theta2_grad = *(delta3,a2t');

    # normalize and regularize gradients
    Theta2_grad = Theta2_grad/m + (lambda/m)* hcat(zeros(size(Theta2, 1), 1), Theta2[:,2:end]);
    Theta1_grad = Theta1_grad/m + (lambda/m)* hcat(zeros(size(Theta1, 1), 1), Theta1[:,2:end]);
    
    # unroll
    grad = vcat(Theta1_grad[:], Theta2_grad[:])
    end

## gradient check

In [ ]:
# calculate gradients using auto differentiation and compare with gradients calculated using backpropagation
# this check is computationally expensive, therefore use on small networks only
# using ForwardDiff
# g = a1 -> ForwardDiff.gradient(nnCostFn, Theta)
# gradient(Theta) - g(Theta) # difference should be negligible

##training

In [ ]:
# Optimization using conjugate gradient (CG) descent algorithm in scipy imported from Python
# using cost function, gradient function and randomly initialized weights
using PyCall
@pyimport scipy.optimize as scop
@time ThetaTrained = scop.fmin_cg(nnCostFn, Theta, fprime=gradient, maxiter=iterations)

##inference

In [ ]:
function predict(ThetaTrained)
    Theta1 = reshape(ThetaTrained[1:hidden_layer_size * (input_layer_size + 1)], hidden_layer_size, (input_layer_size + 1));
    Theta2 = reshape(ThetaTrained[(1 + (hidden_layer_size * (input_layer_size + 1))):end], num_labels, (hidden_layer_size + 1));

    # forward pass
    z2 = *(Theta1, a1test') # input to hidden layer
    a2 = zigmoid(z2) # hidden layer activations
    a2 = [ones(mtest) a2'] # adding bias units
    z3 = *(Theta2, a2') # input to output layer
    a3 = zigmoid(z3') # output layer activations 

    val, p = findmax(a3', 1) # find index holding the highest probability
    (sum(p%10 .== ytest%10) /mtest)*100 # returns prediction accuracy
end

In [ ]:
predict(ThetaTrained)